In [1]:
#importing necessary libraries
from vtk import *
import plotly.colors as plc
import numpy as np
import plotly.graph_objects as go
from IPython.display import display
from ipywidgets import interact, FloatSlider, HBox, VBox
from ipywidgets import Button
#read the image data
reader = vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')  
reader.Update()
image_data = reader.GetOutput()
scalar_data = np.array(image_data.GetPointData().GetArray('ImageFile')).reshape(image_data.GetDimensions(), order='F')
min_scalar_value = np.min(scalar_data)
max_scalar_value = np.max(scalar_data)
# Define the colormap plasma
colormap = plc.sequential.Plasma
#set the isovalue
isovalue=0.0
# Normalize the data value to range between 0 and 1
normalized_value = (isovalue - min_scalar_value) / (max_scalar_value - min_scalar_value)

# Convert the normalized value to an index
index = int(normalized_value * (len(colormap) - 1))

# Get the color corresponding to the index from the colormap
color = colormap[index]
    
# Create vertices
x, y, z = np.indices(scalar_data.shape)


  
#create histogram
histogram = go.Histogram(x=scalar_data.flatten(), opacity=0.7, marker=dict(color='blue'), xbins=dict(
start=-1.0,
end=1.0,
size=0.05
    ))
# Create isosurface
isosurface = go.Isosurface(
    x=x.flatten(),
    y=y.flatten(),
    z=z.flatten(),
    value=scalar_data.flatten(),  # Pass the filtered values
    isomin=isovalue,
    isomax=isovalue,
    opacity=0.8,
    colorscale=[[0, color], [0.5, color], [1, color]],
    surface_count=1,
    showscale=False 
)



g1 = go.FigureWidget(data=[isosurface],layout = go.Layout(
    scene=dict(
        xaxis=dict(showticklabels=False),
        yaxis=dict(showticklabels=False),
        zaxis=dict(showticklabels=False),
        aspectratio=dict(x=1.5, y=1.5, z=1.5)
    )
)
)

g2= go.FigureWidget(data=[histogram])
#function to update plots on changing slider value
def slider_update(change):
        isovalue = change.new
        normalized_value = (isovalue - min_scalar_value) / (max_scalar_value - min_scalar_value)
        index = int(normalized_value * (len(colormap) - 1))
        color = colormap[index]
        with g1.batch_update():
            g1.data[0].isomin=isovalue
            g1.data[0].isomax=isovalue
            g1.data[0].colorscale=[[0, color], [0.5, color], [1, color]]
            
        with g2.batch_update():
            g2.data[0].xbins=dict(
                start=isovalue-0.25,  # Start value of the first bin
                end=isovalue+0.25,    # End value of the last bin
                size=0.02   # Width of each bin
            )
#function to reset plots       
def reset_update(change):
    isovalue = 0.0
    slider.value=0.0
    normalized_value = (isovalue - min_scalar_value) / (max_scalar_value - min_scalar_value)
    index = int(normalized_value * (len(colormap) - 1))
    color = colormap[index]

    with g1.batch_update():
        g1.data[0].isomin=isovalue
        g1.data[0].isomax=isovalue
        g1.data[0].colorscale=[[0, color], [0.5, color], [1, color]]
    with g2.batch_update():
        g2.data[0].xbins=dict(
            start=-1.0,  # Start value of the first bin
            end=1.0,    # End value of the last bin
            size=0.05   # Width of each bin
        )


#set the slider 
#slider starts from min_
slider = FloatSlider(value=0.0, min=min_scalar_value, max=max_scalar_value, step=0.01, description='Isovalue:', continuous_update = False)
# Observing slider value changes
slider.observe(slider_update, names="value") 
# Button to reset plots
reset_button = Button(description='Reset')
reset_button.on_click(reset_update)
# Arrange widgets
#horizontal container of slider and reset button
container=HBox([g1,g2])
#horizontal container of isosurface and histogram
container1=HBox([slider,reset_button])
VBox([container1, container])
